In [ ]:
import pandas as pd
import plotly.express as px
import plotly as plotly

def reformat_grades(rate):
    if(rate.startswith('LE') or rate.startswith('LT') or rate.startswith('GE')):
        return rate[2:]
    if('-' in rate):
        return rate[0:rate.index('-')]
    return rate

In [ ]:
df_grad_rates = pd.read_csv("../Data/18_19_Graduation_Rates.csv")
df_locations = pd.read_csv("../Data/school_geo_data.csv")
df_state_abbrevs = pd.read_csv("../Data/state_mapping.csv")

In [ ]:
df_grad_rates = df_grad_rates[['STNAM','LEAID','SCHNAM','ALL_RATE_1819']] #only need state name, school district, school name, and graduation rate for now
df_grad_rates = df_grad_rates[df_grad_rates.ALL_RATE_1819 != 'PS'] #drop all rows for schools where data was protected

In [ ]:
df_grad_rates['ALL_RATE_1819'] = df_grad_rates['ALL_RATE_1819'].apply(reformat_grades) #remove letters and symbols from graduation rates

In [ ]:
#Replace state names with abbreviations
df_grad_rates['STNAM'] = df_grad_rates['STNAM'].str.lower()
df_state_abbrevs['Name'] = df_state_abbrevs['Name'].str.lower()
df_grad_rates = pd.merge(df_grad_rates,df_state_abbrevs, left_on='STNAM', right_on='Name')

In [ ]:
#drop redundant columns after merge
df_grad_rates = df_grad_rates.drop(['STNAM','Name'], axis=1)
df_grad_rates.head()

In [ ]:
df_plot = pd.merge(df_grad_rates,df_locations,left_on=["SCHNAM","Abbreviation"],right_on=["NAME","STATE"])
df_plot["Grad_Rate"] = df_plot["ALL_RATE_1819"].astype('int32')

In [ ]:
fig = px.scatter_geo(df_plot,lat="LAT",lon="LON", color="Grad_Rate", hover_name="SCHNAM", color_continuous_scale="inferno",scope='usa', width = 1200, height=800)

In [ ]:
import plotly.graph_objects as go # or plotly.express as px
#fig = go.Figure() # or any Plotly Express function e.g. px.bar(...)
# fig.add_trace( ... )
# fig.update_layout( ... )
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

app = dash.Dash()
app.layout = html.Div([
html.P("Filter by Graduation Rate"),
dcc.Slider(min=0, max=100,value=50,id="filter_slide",tooltip={"placement": "bottom", "always_visible": True}),
dcc.Graph(id="main_chart")
])

@app.callback(
    Output('main_chart', 'figure'),
    Input('filter_slide', 'value'))
def update_output(value):
    df = df_plot[df_plot.Grad_Rate < value]
    fig = px.scatter_geo(df,lat="LAT",lon="LON", color="Grad_Rate", hover_name="SCHNAM", color_continuous_scale="inferno",scope='usa',title="Graduation Rate By School", width = 1200, height=800)
    return fig

app.run_server(debug=True, use_reloader=False)